In [11]:
import sys
sys.path.insert(0, '/tf/utils/')

  Preparing metadata (setup.py) ... done
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7779 sha256=84313a235eeab87a254aeda2d27909554972d888c4cd5393a6a4617231a4b80e
  Stored in directory: /root/.cache/pip/wheels/62/35/75/c07f0861a60fb8aacf44fdd5c8c214a224a6c9edb4a4e1402f
Successfully built pystoi

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [12]:
import os

import numpy as np

# Definir o nível de log do TensorFlow para ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Activation, Dropout, Conv1D, Conv1DTranspose, MaxPooling1D, concatenate, Concatenate, LSTM, Dense, SimpleRNN, BatchNormalization, Dropout, BatchNormalization, Add, Flatten, UpSampling1D, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LayerNormalization, Dropout, MultiHeadAttention, Conv2D, Conv2DTranspose, MaxPooling2D, Permute, Reshape, AveragePooling2D, GlobalAveragePooling2D, multiply, GlobalAveragePooling1D, Multiply
from tensorflow.keras import backend as K
from keras.callbacks import Callback

# import sounddevice as sd
from scipy.signal import butter, filtfilt, stft, istft
from scipy.signal import resample
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.io import wavfile
from glob2 import glob
from tqdm import tqdm
from pesq import pesq
from pystoi import stoi
import matplotlib.pyplot as plt
import sys


from utils import load_wav, generate_white_gaussian_noise, add_white_gaussian_noise, undersample_signal_with_antialiasing, somar_sinais, calculate_snr, save_sound_to_wav
import matplotlib.pyplot as plt
from sound import Sound

In [13]:
base_shape_size = 2000

In [14]:
sound_base = Sound('./Base/', './ESC-50-master/audio/', base_shape_size)

Loading noise files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:09<00:00, 211.30it/s]


In [15]:
def calcular_componentes_fourier(sinal):
    if len(sinal.shape) == 3 and sinal.shape[2] == 1:
        sinal = sinal.reshape(sinal.shape[0], sinal.shape[1])

    componentes = np.fft.fft(sinal)
    # Obter a magnitude máxima dos componentes
    # max_magnitude = np.max(np.abs(componentes), axis=1).reshape(-1, 1)
    # Normalizar os componentes para o intervalo [-1, 1]
    max_magnitude = 1
    # componentes_normalizados = componentes / max_magnitude
    res = np.stack((np.real(componentes), np.imag(componentes)), axis=-1)

    return np.array(res), max_magnitude

def reconstruir_sinal(componentes, max_magnitude):
    comp = componentes[:, :, 0] + 1j * componentes[:, :, 1]
#     print(comp.shape)
    sinal_reconstruido = np.fft.ifft(comp) * max_magnitude
    return np.real(sinal_reconstruido)

In [16]:
class DataGenerator:
    def __init__(self, sound_files, noise_files):
        self.sound_files = sound_files
        self.noise_files = noise_files
        self.MIN_NOISE_LEVEL = 5
        self.MAX_NOISE_LEVEL = 30
        
        self.model_pre = load_model('modelo-pre-fourier-09-06-2023.h5')

    def generate_sample_pesq(self, batch_size=32):
        while True:
            # Carrega um lote de sons
            sound_batch_choices = np.random.choice(self.sound_files.shape[0], size=batch_size, replace=False)
            sound_batch = self.sound_files[sound_batch_choices]
            
            # Carrega um lote de ruídos
            noise_batch_choices = np.random.choice(self.noise_files.shape[0], size=batch_size, replace=False)
            noise_batch = self.noise_files[noise_batch_choices]
            
            x_train = []
            y_train = []
            
            # Adiciona ruído a cada som e calcula a nota PESQ
            for sound, noise in zip(sound_batch, noise_batch):
#                 sound = self.normalize_array(sound)
#                 noise = self.normalize_array(noise)
                
                sr = np.random.randint(-5, 90, size=(1,)[0])
                noisy_sound = somar_sinais(sound, noise, sr)
#                 noisy_sound = self.normalize_array(noisy_sound)
                
                noisy_sound = add_white_gaussian_noise(noisy_sound, np.random.randint(7, 90, size=(1,)[0]))
                noisy_sound = np.clip(noisy_sound, -1, 1)
#                 noisy_sound = self.normalize_array(noisy_sound)

                comp, _ = calcular_componentes_fourier(noisy_sound.reshape(-1, base_shape_size))
                original_sound, _ = calcular_componentes_fourier(sound.reshape(-1, base_shape_size))

                filtered = self.model_pre.predict(comp.reshape(-1, base_shape_size, 2), verbose=0)
                filtered_sound = reconstruir_sinal(filtered, 1)
                
#                 print(sound.shape)
#                 print(noisy_sound.shape)
#                 print(comp.shape)
#                 print(original_sound.shape)
#                 print(filtered.shape)
#                 print(filtered_sound.shape)
                
                
                # Calcula a nota PESQ
                try:
                    pesq_score = pesq(8000, sound, filtered_sound.reshape(-1), 'nb')
                except:
                    continue
                valor_min = -0.6
                valor_max = 4.6
                pesq_score = (pesq_score - valor_min) / (valor_max - valor_min)
                
                # Formata dados para treinamento
#                 original_sound = sound #.reshape((base_shape_size, 1))
#                 d_sound = noisy_sound #.reshape((base_shape_size, 1))
                
#                 comp, mmm = calcular_componentes_fourier(r_noisy_sound)
#                 filtered = modelo_pre.predict(comp.reshape(-1, base_shape_size, 2))
                
#                 original_sound, ms = calcular_componentes_fourier(original_sound)
#                 d_sound, md = calcular_componentes_fourier(d_sound)
                
#                 # Formata dados para treinamento
#                 original_sound = sound.reshape((base_shape_size, 1))
#                 d_sound = noisy_sound.reshape((base_shape_size, 1))
                
                merged_array = np.concatenate((original_sound.reshape(base_shape_size, 2), filtered.reshape(base_shape_size, 2)), axis=-1)
#                 print(merged_array.shape)
                # Adiciona o exemplo ao lote de treinamento
                x_train.append(merged_array)
                y_train.append(pesq_score)
            
            yield np.array(x_train), np.array(y_train)

## Modelo para a PESQ

1. Uma função `cnn()` é definida para construir a arquitetura do modelo CNN.
2. O modelo é criado chamando `cnn()`, e um resumo do modelo é impresso.
3. Instâncias de `DataGenerator` são criadas para os dados de treinamento e validação.
4. O modelo é compilado com o otimizador 'adam' e a função de perda 'mse' (erro quadrático médio).
5. Parâmetros de treinamento são definidos, como tamanho do lote e etapas por época.
6. O treinamento começa com um loop que itera três vezes para três épocas.
7. Para cada época, um novo lote de validação é gerado usando o gerador de dados de validação.
8. O modelo é treinado usando o gerador de dados de treinamento, com as etapas por época, épocas, dados de validação e um callback para visualização de perda.
9. Após o treinamento, o modelo é salvo em um arquivo chamado 'pesq-generator-16.h5'.

In [17]:
def cnn(input_layer, kernel_size=3):
    # Define the input shape
#     input_shape = (base_shape_size, 2)

    # Define the model architecture
#     input_layer = Input(shape=input_shape)

    conv1 = Conv1D(16, kernel_size, activation='relu')(input_layer)
    conv1 = Conv1D(16, kernel_size, activation='relu')(conv1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(32, kernel_size, activation='relu')(pool1)
    conv2 = Conv1D(32, kernel_size, activation='relu')(conv2)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    
    conv3 = Conv1D(64, kernel_size, activation='relu')(pool2)
    conv3 = Conv1D(64, kernel_size, activation='relu')(conv3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)

    conv4 = Conv1D(128, kernel_size, activation='relu')(pool3)
    conv4 = Conv1D(128, kernel_size, activation='relu')(conv4)
    pool4 = MaxPooling1D(pool_size=2)(conv4)
    
    conv5 = Conv1D(256, kernel_size, activation='relu')(pool4)
    conv5 = Conv1D(256, kernel_size, activation='relu')(conv5)
    pool5 = MaxPooling1D(pool_size=2)(conv4)
    
    conv6 = Conv1D(512, kernel_size, activation='relu')(pool5)
    conv6 = Conv1D(512, kernel_size, activation='relu')(conv6)
    pool6 = MaxPooling1D(pool_size=2)(conv6)

    flatten = Flatten()(pool6)
    dense1 = Dense(256, activation='relu')(flatten)
    dense1 = Dense(256, activation='relu')(dense1)
#     output_layer = Dense(1, activation='sigmoid')(dense1)

#     model = Model(inputs=input_layer, outputs=output_layer)
    return dense1

In [18]:
# Crie uma instância do DataGenerator
data_generator_train = DataGenerator(sound_base.train_X, sound_base.noise_sounds)
data_generator_val = DataGenerator(sound_base.val_X, sound_base.noise_sounds)

In [19]:
input_shape = (base_shape_size, 4)

input_layer = Input(shape=input_shape)

in1 = cnn(input_layer, kernel_size=3)
# in2 = cnn(input_layer, kernel_size=9)
# in3 = cnn(input_layer, kernel_size=17)
# in4 = cnn(input_layer, kernel_size=35)
# in5 = cnn(input_layer, kernel_size=71)
# in6 = cnn(input_layer, kernel_size=143)

# den = concatenate([in1, in2, in3, in4])
# den = concatenate([in1, in6])

# output = Dense(128, activation='relu')(in1)
# output = Dense(64, activation='relu')(output)

output_layer = Dense(1, activation='sigmoid')(in1)

model = Model(inputs=input_layer, outputs=output_layer)

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2000, 4)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1998, 16)          208       
                                                                 
 conv1d_1 (Conv1D)           (None, 1996, 16)          784       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 998, 16)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 996, 32)           1568      
                                                                 
 conv1d_3 (Conv1D)           (None, 994, 32)           3104      
                                                             

In [21]:
model = load_model('chech.h5')

In [22]:
# model = cnn()
# model.summary()
model.compile(optimizer='adam', loss='mse',)

batch_size = 128
steps_per_epoch = len(sound_base.train_X) // batch_size

print('Starting training')

for epoch in range(8):
    print(f"Epoch {epoch + 1}")
    
    # Gera um novo lote de validação para cada época
    validation_batch = next(data_generator_val.generate_sample_pesq(batch_size=1024))
    x_val, y_val = validation_batch
        
    model.fit(data_generator_train.generate_sample_pesq(batch_size=batch_size),
              steps_per_epoch=steps_per_epoch,
              epochs=1,
              validation_data=(x_val, y_val),
#               callbacks=[PlotLossesCallback()]
            )

Starting training
Epoch 1
254/254 [==============================] - 2009s 8s/step - loss: 0.0051 - val_loss: 0.0046


KeyboardInterrupt: 

In [23]:
model.save('chech.h5')